In [25]:
import requests
import json
from bs4 import BeautifulSoup
import dateparser
import pandas as pd

In [26]:
df_all_rates = pd.DataFrame()
df_all_rates['date'] = pd.date_range('20000101',periods = 6000)

In [27]:
VAL_NM_RQ_dict = {'USD' : 'R01235', 'CNY' : 'R01375', 'EUR': 'R01239'}

for valut in VAL_NM_RQ_dict.values():
    
    cbr_rate_api_res = requests.get("http://www.cbr.ru/scripts/XML_dynamic.asp", params = {
    "date_req1": "01/01/2000",
    "date_req2": "20/12/2023",
    "VAL_NM_RQ": valut})
    cbr_rate_api_res.raise_for_status()
    
    resp_text = cbr_rate_api_res.text.replace('<?xml version="1.0" encoding="windows-1251"?>', '<?xml version="1.0"?>')
    resp_soup = BeautifulSoup(resp_text, "xml")
    val_curs_soup = resp_soup.find("ValCurs")
    val_curs_records = val_curs_soup.find_all("Record")
    print(f"{len(val_curs_records)} records found for {valut}")
    
    rate_list = []

    for record in val_curs_records:
        date_str = record["Date"]
        nom_str = record.find("Nominal").get_text()
        val_str = record.find("Value").get_text()

        nom = float(nom_str.replace(",", "."))
        val = float(val_str.replace(",", "."))

        rate = val / nom
        date = dateparser.parse(date_str)

        record_dict = {
            "date": date,
            f"rate_{valut}": rate
        }
        rate_list.append(record_dict)
        
    rate_df = pd.DataFrame(rate_list)
        #print(rate_df)
    df_all_rates = df_all_rates.merge(rate_df, on = 'date', how = 'outer')

5946 records found for R01235


C:\Users\user\anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


4443 records found for R01375


C:\Users\user\anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


5946 records found for R01239


C:\Users\user\anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [34]:
df_all_rates

,date,rate_R01235,rate_R01375,rate_R01239
0,2000-01-01,27.0000,3.2610,27.2000
1,2000-01-02,28.5500,3.4490,27.9800
2,2000-01-03,28.6500,3.4620,27.7500
3,2000-01-04,28.6000,3.4380,27.4000
4,2000-01-05,NaN,3.4340,NaN
...,...,...,...,...
7856,2023-12-14,89.8926,12.4993,96.9500
7857,2023-12-15,89.6741,12.5457,97.7377
7858,2023-12-16,89.6966,12.6091,98.4186
7859,2023-12-19,90.4162,12.6578,98.7578
